# Setup

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, './venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Instantiate a brand new model

In [2]:
# tokenizer
from tokenizer import get_tokenizer
tokenizer = get_tokenizer(size = 2048) # size options are 95(character-wise), 128, 256, 512, 1024, 2048 & 4096

# config file
from config import ModelConfig, TrainConfig
cfg = ModelConfig()
cfg.vocab_len = tokenizer.vocab_len
tcfg = TrainConfig()
print(cfg, '\n\n', tcfg)

# model modules
from model import customGPT
model = customGPT(cfg).to(cfg.device)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters\n')
print(model)

ModelConfig(dim=64, vocab_len=2051, device='cpu', num_layers=8, second_resid_norm=False, mlp_hidden_mult=2, mlp_bias=False, mlp_nonlinearity='SiLU', mlp_gated=True, num_q_heads=8, num_kv_heads=2, head_dim=16, theta=10000, max_seq_len=512, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06, max_batch_size=1) 

 TrainConfig(weight_decay=0.02, batch_size=32, max_iters=1000, eval_interval=10, eval_samples=1, checkpoint_interval=None, lr_max=0.1, lr_min=1e-06, warmup_iters=100, final_flat_iters=100, anneal_type='cos', num_restarts=3, T_mult=2)
493.888 K parameters

customGPT(
  (token_embedder): Embedding(2051, 64)
  (layers): ModuleList(
    (0-7): 8 x ResidualLayer(
      (pre_attn_norm): Norm()
      (attn): MQSA(
        (Wq): Linear(in_features=64, out_features=128, bias=False)
        (Wk): Linear(in_features=64, out_features=32, bias=False)
        (Wv): Linear(in_features=64, out_features=32, bias=False)
        (Wo): Linear(in_features=128, out

# Training

In [3]:
import torch
from tools import get_data_loader
from train import scheduler_lambda, train

optimizer = torch.optim.AdamW(model.parameters(), lr = tcfg.lr_max, weight_decay = tcfg.weight_decay)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler_lambda)

train_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='train')
test_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='validation')

In [4]:
if False: # set to true if you'd like to see a graph of the learning rate schedule
    import matplotlib.pyplot as plt
    
    # Generate learning rate values
    lrs = [scheduler_lambda(i) for i in range(tcfg.max_iters)]
    
    # Plot the learning rates
    plt.figure(figsize=(10, 5))
    plt.plot(lrs, label='Learning Rate')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Iteration')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.legend()
    plt.show()

In [5]:
model, optimizer, log_data = train(
    model, 
    tokenizer, 
    cfg, 
    optimizer,
    scheduler,
    tcfg, 
    train_data_loader,
    test_data_loader,
    #log_data: list = None, 
    #detect_anomoly = False # use if you're getting crazy errors about a the gradient being broken
)

  0%|                                                                          | 1/1000 [00:10<2:56:31, 10.60s/it]

step 0000: lr 0.000000, train loss 63.0248, val loss 63.1004, ppl 2536014693363622611949453312, time elapsed: 5.46 seconds


  1%|▊                                                                        | 11/1000 [01:06<1:49:39,  6.65s/it]

step 0010: lr 0.001000, train loss 61.5801, val loss 61.5277, ppl 526206877719226915238182912, time elapsed: 61.35 seconds


  2%|█▌                                                                       | 21/1000 [02:00<1:43:11,  6.32s/it]

...

step 0980: lr 0.000000, train loss 2.6580, val loss 2.8351, ppl 17, time elapsed: 5087.55 seconds


 99%|███████████████████████████████████████████████████████████████████████▎| 991/1000 [1:25:43<00:54,  6.00s/it]

step 0990: lr 0.000000, train loss 2.8126, val loss 2.6469, ppl 14, time elapsed: 5138.50 seconds


100%|███████████████████████████████████████████████████████████████████████| 1000/1000 [1:26:28<00:00,  5.19s/it]

step 0999: lr 0.000000, train loss 2.5850, val loss 2.6339, ppl 14, time elapsed: 5184.22 seconds


# inference test before you decide to save it
if `tcfg.checkpoint_interval != None` then checkpoints have already been saved

In [6]:
from inference import generate
prompt = "Once upon a time"
model.eval()
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 512,
    temperature = 0.7,
    #memory_saver_div = 8,
    #top_p = 0.9,
    #top_k = 32,
)
model.train()
print(output)

Once upon a time, there was a pie to coloved Tim. The little girl named Tim. She was very happy. She would mom saw a little girl named Tim had a big mom. The mom was not good a little book, a brel decided to be be. It was very happy and it was not not earned. She wanted to like to man said, "I am always a scar.
She was so excited to the red and a and on the tople. The bird flew that down. The bird saw down and said, "I have did not in the wise the mom. The mom and the pretty becared and found a little. They are so happy and played with a a gree!" Max and went to the snail and you want to knew she longs. The dress was very good all was very happy. They played with the more together every day. He played Lily was so happy other and from the very sad.
Sue was very happy and it was not know what the wish.
The car your was asked to the ground and go the way the in the park. The cat and down. They were toon to make they was speciall and Lily.
A man said disappogether sking. She saybe were all

# Saving your final model
you DO still need to do this even if you had been saving checkpoints; the final state has not yet been saved

In [7]:
from tools import save_model
save_model(model, cfg, tcfg, log_data)